In [1]:
import sys
import os.path
import pikepdf
import urllib.request
import pandas as pd
from datetime import datetime
from pathlib import Path
#import requests
Path("temp").mkdir(parents=True, exist_ok=True)
Path("data").mkdir(parents=True, exist_ok=True)

In [2]:
if (os.path.exists("data/meta.csv")):
    os.remove("data/meta.csv")

In [4]:
#loop
intiger=1
with open("data/PDFURLs.csv") as urls:
    for pdf_url in urls:

        #Creating Unique filename
        unique = pdf_url.removeprefix("https://bildungsportal.sachsen.de/opal/oer/")
        unique = unique.removesuffix("\n")
        metadict = {"Identifier":unique}
        pfad = "temp/" + unique + ".pdf"
        #Try to download file if not in /temp, catch downloaderrors, gesondert 404
        if (not os.path.exists(pfad)):
            print("downloading:", unique)
            try:
                pdf_filename, headers = urllib.request.urlretrieve(pdf_url, pfad)
            except urllib.error.HTTPError as e:
                print("Error while downloading.")
                if e.code == 404:
                    metadict.update({"Error":'Missing(404)'})
                else:
                    metadict.update({"Error":'Unknown Downloaderror'})
                #save Error to meta.csv
                frame = pd.DataFrame(metadict, [0])
                if (os.path.exists("data/meta.csv")):   
                    oldframe = pd.read_csv("data/meta.csv")
                    newframe = pd.concat([oldframe, frame])
                    newframe.to_csv("data/meta.csv", index=False)           
                else:
                    frame.to_csv("data/meta.csv", index=False)
                metadict.clear
                intiger=intiger+1
                continue
        else:
            pdf_filename = pfad
        #try to open pdf, catch Errors
        try:
            pdf = pikepdf.Pdf.open(pdf_filename)           
        except pikepdf._qpdf.PasswordError:
            print("This pdf requires a password.")
            metadict.update({"Error":'PasswordProtected'})
            frame = pd.DataFrame(metadict, [0])
            if (os.path.exists("data/meta.csv")):   
                oldframe = pd.read_csv("data/meta.csv")
                newframe = pd.concat([oldframe, frame])
                newframe.to_csv("data/meta.csv", index=False)           
            else:
                frame.to_csv("data/meta.csv", index=False)
            metadict.clear
            intiger=intiger+1
            continue
        except:
            print("An Error occured while opening.")
            metadict.update({"Error":'OpeningError'})
            frame = pd.DataFrame(metadict, [0])
            if (os.path.exists("data/meta.csv")):   
                oldframe = pd.read_csv("data/meta.csv")
                newframe = pd.concat([oldframe, frame])
                newframe.to_csv("data/meta.csv", index=False)           
            else:
                frame.to_csv("data/meta.csv", index=False)
            metadict.clear
            intiger=intiger+1
            continue
        
        #Read Metadata from pdfs docinfo
        docinfo = pdf.docinfo

        try:
            docinfo["/Author"]
        except KeyError:
            metadict.update({"Author":""})
        else:
            metadict.update({"Author":str(docinfo["/Author"])})

        try:
            docinfo["/Keywords"]
        except KeyError:
            metadict.update({"Keywords":""})
        else:
            metadict.update({"Keywords":str(docinfo["/Keywords"]).replace("\n", " ")})
            
            
        try:
            docinfo["/CreationDate"]
        except KeyError:
            metadict.update({"Datum der Erstellung":""})
        else:
            creation = str(docinfo["/CreationDate"]).replace("'", "")
            
            if(len(creation) > 12):
                if(creation[0] != 'D'):
                    creation = 'D:' + creation
                if(len(creation) >=17):
                    if(creation[16]== 'Z'):
                        creation = creation.removesuffix("0000")
                elif(len(creation) == 16):
                    creation = creation + 'Z'
                try:
                    date = datetime.strptime(creation, "D:%Y%m%d%H%M%S%z")
                except ValueError:
                    metadict.update({"Datum der Erstellung":""})
                    #print("Date Format: ", creation, "\n Maps to:", "unknown")
                else:
                    metadict.update({"Datum der Erstellung":date.strftime("%Y/%m/%d")}) 
                    #print("Known Date Format: ", creation, "\n Maps to:", date)
            else:
                #print("Date to short, dropping: ", creation)
                metadict.update({"Datum der Erstellung":""})
    
        try:
            docinfo["/Title"]
        except KeyError:
            metadict.update({"Titel":""})
        else:
            metadict.update({"Titel":str(docinfo["/Title"])})

            
        metadict.update({"Seitenzahl":len(pdf.pages)})

        
        if (pdf.pages[0].trimbox[2] - pdf.pages[0].trimbox[0]) < (pdf.pages[0].trimbox[3] - pdf.pages[0].trimbox[1]):
            metadict.update({"Format":"Hochformat"})
        else:
            metadict.update({"Format":"Querformat"})
 #old Code zur menschlichen Lesbarkeit der Dateigröße, panda.cut() bietet eine voraussichlich bessere Umsetzung   
#        size = os.stat(pdf_filename).st_size
#        if (size > 1000000000):
#            size = size / 1000000000
#            sizestr =str(round(size, 1)) + " GB"
#        elif (size > 1000000):                    
#            size = size / 1000000
#            sizestr =str(round(size, 1)) + " MB"
#        elif (size > 1000):
#            size = size / 1000
#            sizestr =str(round(size, 1)) + " kB"   
#        else:
#            sizestr =str(round(size, 1)) + " B"
 #       metadict.update({"Dateigröße":str(sizestr)})
        metadict.update({"Dateigröße":os.stat(pdf_filename).st_size})

    
        frame = pd.DataFrame(metadict, [0])
        if (os.path.exists("data/meta.csv")):   
            oldframe = pd.read_csv("data/meta.csv")
            newframe = pd.concat([oldframe, frame])
            newframe.to_csv("data/meta.csv", index=False)
            
        else:
            frame.to_csv("data/meta.csv", index=False)
        metadict.clear
        pdf.close()
        print("finshed:", intiger)
        intiger=intiger+1

finshed: 1
finshed: 2
finshed: 3
finshed: 4
finshed: 5
finshed: 6
finshed: 7
finshed: 8
finshed: 9
finshed: 10
finshed: 11
finshed: 12
finshed: 13
finshed: 14
finshed: 15
finshed: 16
finshed: 17
finshed: 18
finshed: 19
finshed: 20
finshed: 21
downloading: 7zMn-et47v7I
Error while downloading.
finshed: 23
downloading: 1FDZEIhl8k4M
Error while downloading.
finshed: 25
finshed: 26
finshed: 27
finshed: 28
finshed: 29
finshed: 30
finshed: 31
finshed: 32
finshed: 33
finshed: 34
finshed: 35
finshed: 36
finshed: 37
finshed: 38
finshed: 39
finshed: 40
finshed: 41
finshed: 42
finshed: 43
finshed: 44
finshed: 45
finshed: 46
finshed: 47
finshed: 48
finshed: 49
finshed: 50
finshed: 51
finshed: 52
finshed: 53
finshed: 54
finshed: 55
finshed: 56
finshed: 57
finshed: 58
finshed: 59
finshed: 60
finshed: 61
finshed: 62
finshed: 63
finshed: 64
finshed: 65
finshed: 66
finshed: 67
finshed: 68
finshed: 69
finshed: 70
finshed: 71
finshed: 72
finshed: 73
finshed: 74
finshed: 75
finshed: 76
finshed: 77
finshed


KeyboardInterrupt



D:1910 11 10 31 95 255 
 D:2004 26 15202621Z 

In [ ]:
D:191011103195255 